# Produce Driver Standings

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
# race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results.parquet")
# race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")
race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results").filter(f"file_date = '{v_file_date}'")

In [0]:
race_results_df.show(5)

+-------+---------+------------------+-------------------+----------------+----------------+------------------+--------+----+-----------+-----------+------+--------+----------+--------------------+
|race_id|race_year|         race_name|          race_date|circuit_location|     driver_name|driver_nationality|    team|grid|fastest_lap|  race_time|points|position| file_date|        created_date|
+-------+---------+------------------+-------------------+----------------+----------------+------------------+--------+----+-----------+-----------+------+--------+----------+--------------------+
|    346|     2010|British Grand Prix|2010-07-11 12:00:00|     Silverstone|  Lewis Hamilton|           British| McLaren|   4|         43|     +1.360|  18.0|       2|2021-03-21|2023-06-17 19:10:...|
|    346|     2010|British Grand Prix|2010-07-11 12:00:00|     Silverstone|   Robert Kubica|            Polish| Renault|   6|         12|         \N|   0.0|    null|2021-03-21|2023-06-17 19:10:...|
|    346| 

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as f

In [0]:
driver_standings_df = race_results_df.groupBy("race_year", "driver_name", "driver_nationality", "team") \
                        .agg(f.sum("points").alias("total_points"), f.count(f.when(f.col("position") == 1, True)).alias("wins"))

In [0]:
display(driver_standings_df.filter("race_year = 2020"))

race_year,driver_name,driver_nationality,team,total_points,wins


In [0]:
driver_rank_spec = Window.partitionBy("race_year").orderBy(f.desc("total_points"), f.desc("wins"))

In [0]:
final_df = driver_standings_df.withColumn("rank", f.rank().over(driver_rank_spec))

In [0]:
display(final_df.filter("race_year = 2020"))

race_year,driver_name,driver_nationality,team,total_points,wins,rank


In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings.parquet")
# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standings")
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_id = src.race_id"
merge_delta_data(final_df, 'f1_presentation', 'driver_standings', presentation_folder_path, merge_condition, 'race_id')